In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup 
import re

In [2]:
url = "https://www.scotrail.co.uk/plan-your-journey/stations-and-facilities"

In [3]:
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
links = []
re_string = r"^\/plan-your-journey\/stations-and-facilities\/([a-z][a-z][a-z])$"
for link in soup.find_all('a'):
    link_text = link.get('href')
    if link_text is not None:
        if re.search(re_string, link_text) is not None:
            links.append(link_text)

In [5]:
station_codes = list(map(lambda x: x[-3:].upper(), links))

In [6]:
urls = [f"https://en.wikipedia.org/wiki/UK_railway_stations_%E2%80%93_{letter}" for letter in "ABCDEFGHIJKLMNOPQRSTUVWXYZ"]
station_info = []


In [7]:
for url in urls:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    rows = soup.find_all("tr")
    print()
    for row in rows:
        cols = row.find_all("a")
        try:
            station_info.append([cols[1].text, cols[2].text[:3]])
        except Exception:
            continue

In [8]:
scottish_stations = list(filter(lambda x: x[1] in station_codes, station_info))

In [9]:
def get_coords(postcode):
    try:
        result = requests.get(f"https://findthatpostcode.uk/postcodes/{postcode.replace(' ', '+')}.json")
        location =  result.json()['data']['attributes']['location']
        return location['lat'], location['lon']
    except Exception:
        print(f"Failed on {postcode}")
        return "FAILED", "FAILED"

In [10]:
for station in scottish_stations:
    lat, lon = get_coords(station[0])
    station.append(lat)
    station.append(lon)

Failed on EH48 3BW
Failed on G69 1AA
Failed on PA9 1XX


In [11]:
df_station = pd.DataFrame([s[:4] for s in scottish_stations], columns = ["Postcode", "CRS", "lat", "lon"])
df_station = pd.DataFrame(station_codes, columns = ["CRS"]).merge(df_station, how = 'left', on = 'CRS')
df_station.head()

,CRS,Postcode,lat,lon
0,ABD,AB11 6RT,57.1441,-2.09878
1,AUR,KY3 0TQ,56.0545,-3.29964
2,AAT,IV23 2QD,57.6097,-4.93896
3,ACN,IV22 2EE,57.5797,-5.07546
4,ACH,IV54 8YU,57.4779,-5.33297


In [12]:
nlc_codes = pd.read_csv("C:\\Users\\User\\Documents\\4. Fourth Year\\Project\\Railway-Fares\\Data\\nlc_codes.csv", dtype = str)
nlc_codes.head()

,Location,CRS,NLC,TIPLOC,STANME,STANOX,NLC Clipped
0,Aachen,NaN,081601,AACHEN,AACHEN,00005,NaN
1,Abbeyhill Junction,NaN,937800,ABHLJN,ABBYHILLJ,04309,9378
2,Abbeyhill Signal E811,NaN,937802,ABHL811,EDSIG811,04311,NaN
3,Abbeyhill Turnback Sidings,NaN,937801,ABHLTB,ABBYHILTB,04308,NaN
4,Abbey Level Crossing (Staffordshire),NaN,130201,NaN,NaN,NaN,NaN


In [13]:
station_master = df_station.merge(nlc_codes, on='CRS', how = 'left')

In [14]:
station_master = station_master[["Postcode", "CRS", "lat", "lon", "Location", "NLC Clipped"]]
station_master = station_master.rename(columns = {"Location": "Name", "NLC Clipped":"NLC"})

In [15]:
station_master.to_csv("C:\\Users\\User\\Documents\\4. Fourth Year\\Project\\Railway-Fares\\Data\\Cleansed Data\\station.csv", index = False)